In [1]:
import re
from datasets import load_dataset

DATASET_NAME = "verifiers-for-code/gold_tester_llama"
COL_TO_CLEAN = "gold_plans_llama-3-70B"
COL_CLEANED_NAME = "cleaned-" + COL_TO_CLEAN

dataset = load_dataset(DATASET_NAME, split="test")

def extract_plan(text):
    start_match = re.search(r'<plan>', text)
    if start_match:
        start_index = start_match.end()
        end_match = re.search(r'</plan>', text[start_index:])
        
        if end_match:
            end_index = start_index + end_match.start()
            plan = text[start_index:end_index].strip()
        else:
            # If no closing tag, include everything after <plan>
            plan = text[start_index:].strip()
        
        return plan.split('\n')
    else:
        print("No plan found in text")
    return []

def insert_plan_into_docstring(prompt, plan):
    # Find the end of the docstring
    docstring_start = prompt.find('"""')
    docstring_end = prompt.find('"""', docstring_start + 3)
    
    # If there's no docstring or it's malformed, return the original prompt
    if docstring_end == -1:
        return prompt
    
    # Get the indentation of the closing docstring
    closing_indent = prompt.rfind('\n', 0, docstring_end) + 1
    indent = prompt[closing_indent:docstring_end].replace('"""', '')
    
    # Prepare the plan text with proper indentation, without "Plan:" header
    plan_text = "\n\n" + "\n".join(indent + line for line in plan)
    
    return prompt[:docstring_end] + plan_text + prompt[docstring_end:]

# Apply the extraction and insertion to the dataset
def process_item(item):
    plan = extract_plan(item[COL_TO_CLEAN])
    new_prompt = insert_plan_into_docstring(item['prompt'], plan)
    item[COL_CLEANED_NAME] = new_prompt  # Add the new column to the item
    return item  # Return the entire item with the new column

dataset = dataset.map(process_item)

# Print a sample to verify
print(dataset[56][COL_CLEANED_NAME])

/home/vaibhav/miniconda3/envs/inference/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 164/164 [00:00<00:00, 8019.42 examples/s]



def correct_bracketing(brackets: str):
    """ brackets is a string of "<" and ">".
    return True if every opening bracket has a corresponding closing bracket.

    >>> correct_bracketing("<")
    False
    >>> correct_bracketing("<>")
    True
    >>> correct_bracketing("<<><>>")
    True
    >>> correct_bracketing("><<>")
    False
    

    Action Plan:
    1. Initialize a counter to track the depth of opening brackets.
    2. Iterate through each character in the input brackets string:
       a. If the character is an opening bracket "<", increment the depth counter.
       b. If the character is a closing bracket ">", decrement the depth counter.
       c. If the depth counter becomes negative at any point, immediately return False (indicating incorrect bracketing).
    3. After iterating through the entire string, check if the depth counter is 0.
       a. If it is, return True (indicating correct bracketing).
       b. If it's not, return False (indicating incorrect bracketi

In [2]:
dataset.column_names

['task_id',
 'prompt',
 'canonical_solution',
 'test',
 'entry_point',
 'gold_plans_llama-3-70B',
 'cleaned-gold_plans_llama-3-70B']

In [3]:
dataset.push_to_hub(DATASET_NAME, split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  2.28it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/verifiers-for-code/gold_tester_llama/commit/be41a9747a88f5f4ca4adff4e8e4c009d800bf91', commit_message='Upload dataset', commit_description='', oid='be41a9747a88f5f4ca4adff4e8e4c009d800bf91', pr_url=None, pr_revision=None, pr_num=None)